In [1]:
%pip -q install  numpy sympy matplotlib ipywidgets pandas plotly anywidget

Note: you may need to restart the kernel to use updated packages.


In [2]:
from gu_toolkit import *

In [3]:
from gu_toolkit.SmartFigure import *

# Plotting a Function with Plotly (Jupyter Notebook)

This notebook shows how to:
1. Define a mathematical function `f(x)`.
2. Sample it on a grid of `x` values.
3. Plot the result interactively using Plotly.

Plotly plots are interactive: you can zoom, pan, and hover to see values.


## 2) Define the function

We'll define a symbolic function `F(x)`. 

In [4]:
@NamedFunction
def F(x):
    return x**2+1
display(Latex(f"${latex(F(x))} =  {latex(F(x).rewrite("expand_definition"))} $"))

<IPython.core.display.Latex object>

In [5]:
F_f_numpy=numpify(F(x))
F_f_numpy?

Signature: F_f_numpy(x)
Docstring:
Auto-generated NumPy function from SymPy expression.

    expr: x**2 + 1
    args: ['x']

    Source:
    def _generated(x):
x = numpy.asarray(x)
return x**2 + 1
File:      Dynamically generated function. No source code available.
Type:      function

## 4) Create the Plotly figure

We'll add a line trace for the graph of `y = F(x)` and label the axes.


In [6]:
fig=SmartFigure()

In [7]:
fig 

OneShotOutput()

In [8]:
fig.title="Figure 1"

## 5) Adding parameters

In [9]:
fig.add_param(a[1])

## Creating the plot

In [10]:
fig.plot(x, sin(2*pi*x*a[1]), parameters=[a[1]], id="sin")

Parameters: [a_1]


In [11]:
fig.plot(x, cos(2*pi*x*a[1]), parameters=[a[1]], id="cos")

Parameters: [a_1]


# Prototyping and development

In [12]:
# Stop automatic execution
raise NotImplementedError("stop")

NotImplementedError: stop

In [ ]:
a[1].free_symbols

In [ ]:
a[1]

## Widget usage

In [ ]:
# A. Create the widget
slider_widget = SmartFloatSlider(
    value=1.0, 
    min=0.0, 
    max=10.0, 
    step=0.1, 
    description='Param $\\alpha$:'
)

# B. (Optional) Listen for changes
def on_change(change):
    # This prints to the log/output area
    print(f"Value updated: {change['new']}")

slider_widget.observe(on_change, names='value')

# C. Display it
display(slider_widget)

## Input conversion

In [ ]:
print(InputConvert("1.0",truncate=False))
print(InputConvert("6/4",truncate=False))
print(InputConvert("6/4",truncate=False))
print(InputConvert("1+6/4",truncate=False))
print(InputConvert("1+6/4+I*(2/3+5)",dest_type=complex))


## Widgets prototype

In [ ]:
# FloatSliderWithControls
import ipywidgets as widgets
from IPython.display import display, clear_output
import traitlets

class FloatSliderWithControls(widgets.VBox):
    """A FloatSlider with reset button, settings menu, and keyboard input support."""
    
    value = traitlets.Float(default_value=0.0)
    
    def __init__(self, value=0.0, min=0.0, max=1.0, step=0.1, description='Value:', **kwargs):
        super().__init__(**kwargs)
        
        # Store initial values for reset functionality
        self._initial_value = value
        self._initial_min = min
        self._initial_max = max
        self._initial_step = step
        
        # Create main slider
        self.slider = widgets.FloatSlider(
            value=value,
            min=min,
            max=max,
            step=step,
            description=description,
            continuous_update=True,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='80%')
        )
        
        # Create text input for keyboard editing
        self.text_input = widgets.FloatText(
            value=value,
            min=min,
            max=max,
            step=step,
            description='',
            layout=widgets.Layout(width='100px')
        )
        
        # Create reset button
        self.reset_button = widgets.Button(
            description='↺',
            tooltip='Reset to initial value',
            layout=widgets.Layout(width='40px')
        )
        
        # Create settings button
        self.settings_button = widgets.Button(
            description='⚙',
            tooltip='Settings',
            layout=widgets.Layout(width='40px')
        )
        
        # Create settings panel (initially hidden)
        self.settings_panel = widgets.HBox([
            widgets.FloatText(
                value=min,
                description='Min:',
                style={'description_width': '60px'},
                layout=widgets.Layout(width='120px')
            ),
            widgets.FloatText(
                value=max,
                description='Max:',
                style={'description_width': '60px'},
                layout=widgets.Layout(width='120px')
            ),
            widgets.FloatText(
                value=step,
                description='Step:',
                style={'description_width': '60px'},
                layout=widgets.Layout(width='120px')
            )
        ], layout=widgets.Layout(
            display='none',
            border='1px solid #ddd',
            padding='10px',
            margin='5px 0'
        ))
        
        # Settings button style
        self.settings_button.add_class('settings-btn')
        
        # Create top row with slider, text input, and buttons
        self.top_row = widgets.HBox([
            self.slider,
            self.text_input,
            self.reset_button,
            self.settings_button
        ], layout=widgets.Layout(align_items='center'))
        
        # Assemble the widget
        self.children = [self.top_row, self.settings_panel]
        
        # Set up event handlers
        self.reset_button.on_click(self._on_reset)
        self.settings_button.on_click(self._toggle_settings)
        
        # Link slider and text input
        widgets.link((self.slider, 'value'), (self.text_input, 'value'))
        
        # Connect text input changes to update slider
        self.text_input.observe(self._on_text_input_change, 'value')
        
        # Connect settings panel changes
        for i, setting in enumerate(['min', 'max', 'step']):
            self.settings_panel.children[i].observe(
                lambda change, s=setting: self._on_setting_change(s, change),
                'value'
            )
        
        # Link the main value trait to slider value
        traitlets.link((self, 'value'), (self.slider, 'value'))
    
    def _on_reset(self, button):
        """Reset slider to initial value."""
        self.slider.value = self._initial_value
    
    def _toggle_settings(self, button):
        """Toggle visibility of settings panel."""
        if self.settings_panel.layout.display == 'none':
            self.settings_panel.layout.display = 'flex'
        else:
            self.settings_panel.layout.display = 'none'
    
    def _on_text_input_change(self, change):
        """Handle text input changes."""
        if change['new'] is not None:
            # Clamp value to min/max
            value = max(self.slider.min, min(change['new'], self.slider.max))
            self.slider.value = value
    
    def _on_setting_change(self, setting, change):
        """Handle changes to settings panel."""
        if change['new'] is not None:
            if setting == 'min':
                self.slider.min = change['new']
                self.text_input.min = change['new']
                # Update value if needed
                if self.slider.value < change['new']:
                    self.slider.value = change['new']
            elif setting == 'max':
                self.slider.max = change['new']
                self.text_input.max = change['new']
                # Update value if needed
                if self.slider.value > change['new']:
                    self.slider.value = change['new']
            elif setting == 'step':
                self.slider.step = change['new']
                self.text_input.step = change['new']



In [ ]:

    
# Create and display the widget
slider = SmartFloatSlider(
    value=50.0,
    min=0.0,
    max=100.0,
    step=1.0,
    description='Temperature:',
    layout=widgets.Layout(width='100%')
)

# Display the widget
display(slider)
    

## Profiling

In [ ]:
import cProfile, pstats

In [ ]:
profiler = cProfile.Profile()
profiler.enable()

In [ ]:
profiler.disable()
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats(20)  # Prints top 20 time-consuming functions